### Redistricting Indiana

Group 2: Charles Lamb, Connor Cassedy, Heidi Huckabay, and Susan Alrifai

### Library Import

In [1]:
#import libraries
import pulp, numpy as np, pandas as pd
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, lpSum

### Indiana State Data

Data source is US Census data containing 2020 population size by county for the state of Indiana: https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html

In [2]:
#bringing in county and population as list
county_pop = [['Adams', 35811], ['Allen', 385411], ['Bartholomew', 82218], ['Benton', 8720], ['Blackford', 12115], ['Boone', 70814], ['Brown', 15475], ['Carroll', 20304], ['Cass', 37870], ['Clark', 121091], ['Clay', 26457], ['Clinton', 33188], ['Crawford', 10530], ['Daviess', 33391], ['Dearborn', 50680], ['Decatur', 26471], ['DeKalb', 43273], ['Delaware', 111909], ['Dubois', 43635], ['Elkhart', 207054], ['Fayette', 23404], ['Floyd', 80474], ['Fountain', 16474], ['Franklin', 22796], ['Fulton', 20480], ['Gibson', 33017], ['Grant', 66661], ['Greene', 30805], ['Hamilton', 347465], ['Hancock', 79851], ['Harrison', 39649], ['Hendricks', 174792], ['Henry', 48912], ['Howard', 83659], ['Huntington', 36670], ['Jackson', 46425], ['Jasper', 32912], ['Jay', 20478], ['Jefferson', 33147], ['Jennings', 27614], ['Johnson', 161768], ['Knox', 36286], ['Kosciusko', 80247], ['LaGrange', 40444], ['Lake', 498707], ['LaPorte', 112422], ['Lawrence', 45015], ['Madison', 130141], ['Marion', 977213], ['Marshall', 46096], ['Martin', 9806], ['Miami', 35972], ['Monroe', 139717], ['Montgomery', 37941], ['Morgan', 71785], ['Newton', 13825], ['Noble', 47461], ['Ohio', 5930], ['Orange', 19872], ['Owen', 21326], ['Parke', 16154], ['Perry', 19170], ['Pike', 12252], ['Porter', 173208], ['Posey', 25213], ['Pulaski', 12523], ['Putnam', 36728], ['Randolph', 24502], ['Ripley', 28984], ['Rush', 16755], ['Saint_Joseph', 272914], ['Scott', 24383], ['Shelby', 45052], ['Spencer', 19814], ['Starke', 23370], ['Steuben', 34448], ['Sullivan', 20820], ['Switzerland', 9753], ['Tippecanoe', 186249], ['Tipton', 15355], ['Union', 7087], ['Vanderburgh', 180136], ['Vermillion', 15447], ['Vigo', 106158], ['Wabash', 30986], ['Warren', 8436], ['Warrick', 63896], ['Washington', 28176], ['Wayne', 66555], ['Wells', 28187], ['White', 24691], ['Whitley', 34190]]

In [3]:
#setting up a dataframe
df0 = pd.DataFrame(county_pop,columns=['county','pop'])

In [4]:
#checking we have the appropriate number of counties in the dataframe
#per the assignment we expect to see 92
county_cnt = len(df0['county'].unique())
print(county_cnt)

92


### Adjacent Pair Setup

Based on data from the following: https://www2.census.gov/geo/docs/reference/county_adjacency.txt

Some pre-proceesing was performed.  Pairs were found only for the state of Indiana.  And some many adjustment to county name conventions was made to be consistent with the rest of data used in this file

In [5]:
county_pair_list= [['Adams','Allen'],['Adams','Jay'],['Adams','Wells'],['Allen','Adams'],['Allen','DeKalb'],['Allen','Huntington'],['Allen','Noble'],['Allen','Wells'],['Allen','Whitley'],['Bartholomew','Brown'],['Bartholomew','Decatur'],['Bartholomew','Jackson'],['Bartholomew','Jennings'],['Bartholomew','Johnson'],['Bartholomew','Shelby'],['Benton','Jasper'],['Benton','Newton'],['Benton','Tippecanoe'],['Benton','Warren'],['Benton','White'],['Blackford','Delaware'],['Blackford','Grant'],['Blackford','Jay'],['Blackford','Wells'],['Boone','Clinton'],['Boone','Hamilton'],['Boone','Hendricks'],['Boone','Marion'],['Boone','Montgomery'],['Brown','Bartholomew'],['Brown','Jackson'],['Brown','Johnson'],['Brown','Monroe'],['Brown','Morgan'],['Carroll','Cass'],['Carroll','Clinton'],['Carroll','Howard'],['Carroll','Tippecanoe'],['Carroll','White'],['Cass','Carroll'],['Cass','Fulton'],['Cass','Howard'],['Cass','Miami'],['Cass','Pulaski'],['Cass','White'],['Clark','Floyd'],['Clark','Jefferson'],['Clark','Scott'],['Clark','Washington'],['Clay','Greene'],['Clay','Owen'],['Clay','Parke'],['Clay','Putnam'],['Clay','Sullivan'],['Clay','Vigo'],['Clinton','Boone'],['Clinton','Carroll'],['Clinton','Hamilton'],['Clinton','Howard'],['Clinton','Montgomery'],['Clinton','Tippecanoe'],['Clinton','Tipton'],['Crawford','Dubois'],['Crawford','Harrison'],['Crawford','Orange'],['Crawford','Perry'],['Crawford','Washington'],['Daviess','Dubois'],['Daviess','Greene'],['Daviess','Knox'],['Daviess','Martin'],['Daviess','Pike'],['Dearborn','Franklin'],['Dearborn','Ohio'],['Dearborn','Ripley'],['Decatur','Bartholomew'],['Decatur','Franklin'],['Decatur','Jennings'],['Decatur','Ripley'],['Decatur','Rush'],['Decatur','Shelby'],['DeKalb','Allen'],['DeKalb','LaGrange'],['DeKalb','Noble'],['DeKalb','Steuben'],['Delaware','Blackford'],['Delaware','Grant'],['Delaware','Henry'],['Delaware','Jay'],['Delaware','Madison'],['Delaware','Randolph'],['Dubois','Crawford'],['Dubois','Daviess'],['Dubois','Martin'],['Dubois','Orange'],['Dubois','Perry'],['Dubois','Pike'],['Dubois','Spencer'],['Dubois','Warrick'],['Elkhart','Kosciusko'],['Elkhart','LaGrange'],['Elkhart','Marshall'],['Elkhart','Noble'],['Elkhart','Saint_Joseph'],['Fayette','Franklin'],['Fayette','Henry'],['Fayette','Rush'],['Fayette','Union'],['Fayette','Wayne'],['Floyd','Clark'],['Floyd','Harrison'],['Floyd','Washington'],['Fountain','Montgomery'],['Fountain','Parke'],['Fountain','Tippecanoe'],['Fountain','Vermillion'],['Fountain','Warren'],['Franklin','Dearborn'],['Franklin','Decatur'],['Franklin','Fayette'],['Franklin','Ripley'],['Franklin','Rush'],['Franklin','Union'],['Fulton','Cass'],['Fulton','Kosciusko'],['Fulton','Marshall'],['Fulton','Miami'],['Fulton','Pulaski'],['Fulton','Starke'],['Fulton','Wabash'],['Gibson','Knox'],['Gibson','Pike'],['Gibson','Posey'],['Gibson','Vanderburgh'],['Gibson','Warrick'],['Grant','Blackford'],['Grant','Delaware'],['Grant','Howard'],['Grant','Huntington'],['Grant','Madison'],['Grant','Miami'],['Grant','Tipton'],['Grant','Wabash'],['Grant','Wells'],['Greene','Clay'],['Greene','Daviess'],['Greene','Knox'],['Greene','Lawrence'],['Greene','Martin'],['Greene','Monroe'],['Greene','Owen'],['Greene','Sullivan'],['Hamilton','Boone'],['Hamilton','Clinton'],['Hamilton','Hancock'],['Hamilton','Madison'],['Hamilton','Marion'],['Hamilton','Tipton'],['Hancock','Hamilton'],['Hancock','Henry'],['Hancock','Madison'],['Hancock','Marion'],['Hancock','Rush'],['Hancock','Shelby'],['Harrison','Crawford'],['Harrison','Floyd'],['Harrison','Washington'],['Hendricks','Boone'],['Hendricks','Marion'],['Hendricks','Montgomery'],['Hendricks','Morgan'],['Hendricks','Putnam'],['Henry','Delaware'],['Henry','Fayette'],['Henry','Hancock'],['Henry','Madison'],['Henry','Randolph'],['Henry','Rush'],['Henry','Wayne'],['Howard','Carroll'],['Howard','Cass'],['Howard','Clinton'],['Howard','Grant'],['Howard','Miami'],['Howard','Tipton'],['Huntington','Allen'],['Huntington','Grant'],['Huntington','Wabash'],['Huntington','Wells'],['Huntington','Whitley'],['Jackson','Bartholomew'],['Jackson','Brown'],['Jackson','Jennings'],['Jackson','Lawrence'],['Jackson','Monroe'],['Jackson','Scott'],['Jackson','Washington'],['Jasper','Benton'],['Jasper','Lake'],['Jasper','LaPorte'],['Jasper','Newton'],['Jasper','Porter'],['Jasper','Pulaski'],['Jasper','Starke'],['Jasper','White'],['Jay','Adams'],['Jay','Blackford'],['Jay','Delaware'],['Jay','Randolph'],['Jay','Wells'],['Jefferson','Clark'],['Jefferson','Jennings'],['Jefferson','Ripley'],['Jefferson','Scott'],['Jefferson','Switzerland'],['Jennings','Bartholomew'],['Jennings','Decatur'],['Jennings','Jackson'],['Jennings','Jefferson'],['Jennings','Ripley'],['Jennings','Scott'],['Johnson','Bartholomew'],['Johnson','Brown'],['Johnson','Marion'],['Johnson','Morgan'],['Johnson','Shelby'],['Knox','Daviess'],['Knox','Gibson'],['Knox','Greene'],['Knox','Pike'],['Knox','Sullivan'],['Kosciusko','Elkhart'],['Kosciusko','Fulton'],['Kosciusko','Marshall'],['Kosciusko','Noble'],['Kosciusko','Wabash'],['Kosciusko','Whitley'],['LaGrange','DeKalb'],['LaGrange','Elkhart'],['LaGrange','Noble'],['LaGrange','Steuben'],['Lake','Jasper'],['Lake','Newton'],['Lake','Porter'],['LaPorte','Jasper'],['LaPorte','Porter'],['LaPorte','Saint_Joseph'],['LaPorte','Starke'],['Lawrence','Greene'],['Lawrence','Jackson'],['Lawrence','Martin'],['Lawrence','Monroe'],['Lawrence','Orange'],['Lawrence','Washington'],['Madison','Delaware'],['Madison','Grant'],['Madison','Hamilton'],['Madison','Hancock'],['Madison','Henry'],['Madison','Tipton'],['Marion','Boone'],['Marion','Hamilton'],['Marion','Hancock'],['Marion','Hendricks'],['Marion','Johnson'],['Marion','Morgan'],['Marion','Shelby'],['Marshall','Elkhart'],['Marshall','Fulton'],['Marshall','Kosciusko'],['Marshall','Pulaski'],['Marshall','Saint_Joseph'],['Marshall','Starke'],['Martin','Daviess'],['Martin','Dubois'],['Martin','Greene'],['Martin','Lawrence'],['Martin','Orange'],['Miami','Cass'],['Miami','Fulton'],['Miami','Grant'],['Miami','Howard'],['Miami','Wabash'],['Monroe','Brown'],['Monroe','Greene'],['Monroe','Jackson'],['Monroe','Lawrence'],['Monroe','Morgan'],['Monroe','Owen'],['Montgomery','Boone'],['Montgomery','Clinton'],['Montgomery','Fountain'],['Montgomery','Hendricks'],['Montgomery','Parke'],['Montgomery','Putnam'],['Montgomery','Tippecanoe'],['Morgan','Brown'],['Morgan','Hendricks'],['Morgan','Johnson'],['Morgan','Marion'],['Morgan','Monroe'],['Morgan','Owen'],['Morgan','Putnam'],['Newton','Benton'],['Newton','Jasper'],['Newton','Lake'],['Noble','Allen'],['Noble','DeKalb'],['Noble','Elkhart'],['Noble','Kosciusko'],['Noble','LaGrange'],['Noble','Steuben'],['Noble','Whitley'],['Ohio','Dearborn'],['Ohio','Ripley'],['Ohio','Switzerland'],['Orange','Crawford'],['Orange','Dubois'],['Orange','Lawrence'],['Orange','Martin'],['Orange','Washington'],['Owen','Clay'],['Owen','Greene'],['Owen','Monroe'],['Owen','Morgan'],['Owen','Putnam'],['Parke','Clay'],['Parke','Fountain'],['Parke','Montgomery'],['Parke','Putnam'],['Parke','Vermillion'],['Parke','Vigo'],['Perry','Crawford'],['Perry','Dubois'],['Perry','Spencer'],['Pike','Daviess'],['Pike','Dubois'],['Pike','Gibson'],['Pike','Knox'],['Pike','Warrick'],['Porter','Jasper'],['Porter','Lake'],['Porter','LaPorte'],['Porter','Starke'],['Posey','Gibson'],['Posey','Vanderburgh'],['Pulaski','Cass'],['Pulaski','Fulton'],['Pulaski','Jasper'],['Pulaski','Marshall'],['Pulaski','Starke'],['Pulaski','White'],['Putnam','Clay'],['Putnam','Hendricks'],['Putnam','Montgomery'],['Putnam','Morgan'],['Putnam','Owen'],['Putnam','Parke'],['Randolph','Delaware'],['Randolph','Henry'],['Randolph','Jay'],['Randolph','Wayne'],['Ripley','Dearborn'],['Ripley','Decatur'],['Ripley','Franklin'],['Ripley','Jefferson'],['Ripley','Jennings'],['Ripley','Ohio'],['Ripley','Switzerland'],['Rush','Decatur'],['Rush','Fayette'],['Rush','Franklin'],['Rush','Hancock'],['Rush','Henry'],['Rush','Shelby'],['Saint_Joseph','Elkhart'],['Saint_Joseph','LaPorte'],['Saint_Joseph','Marshall'],['Saint_Joseph','Saint_Joseph'],['Saint_Joseph','Starke'],['Scott','Clark'],['Scott','Jackson'],['Scott','Jefferson'],['Scott','Jennings'],['Scott','Washington'],['Shelby','Bartholomew'],['Shelby','Decatur'],['Shelby','Hancock'],['Shelby','Johnson'],['Shelby','Marion'],['Shelby','Rush'],['Spencer','Dubois'],['Spencer','Perry'],['Spencer','Warrick'],['Starke','Fulton'],['Starke','Jasper'],['Starke','LaPorte'],['Starke','Marshall'],['Starke','Porter'],['Starke','Pulaski'],['Starke','Saint_Joseph'],['Steuben','DeKalb'],['Steuben','LaGrange'],['Steuben','Noble'],['Sullivan','Clay'],['Sullivan','Greene'],['Sullivan','Knox'],['Sullivan','Vigo'],['Switzerland','Jefferson'],['Switzerland','Ohio'],['Switzerland','Ripley'],['Tippecanoe','Benton'],['Tippecanoe','Carroll'],['Tippecanoe','Clinton'],['Tippecanoe','Fountain'],['Tippecanoe','Montgomery'],['Tippecanoe','Warren'],['Tippecanoe','White'],['Tipton','Clinton'],['Tipton','Grant'],['Tipton','Hamilton'],['Tipton','Howard'],['Tipton','Madison'],['Union','Fayette'],['Union','Franklin'],['Union','Wayne'],['Vanderburgh','Gibson'],['Vanderburgh','Posey'],['Vanderburgh','Warrick'],['Vermillion','Fountain'],['Vermillion','Parke'],['Vermillion','Vigo'],['Vermillion','Warren'],['Vigo','Clay'],['Vigo','Parke'],['Vigo','Sullivan'],['Vigo','Vermillion'],['Wabash','Fulton'],['Wabash','Grant'],['Wabash','Huntington'],['Wabash','Kosciusko'],['Wabash','Miami'],['Wabash','Whitley'],['Warren','Benton'],['Warren','Fountain'],['Warren','Tippecanoe'],['Warren','Vermillion'],['Warrick','Dubois'],['Warrick','Gibson'],['Warrick','Pike'],['Warrick','Spencer'],['Warrick','Vanderburgh'],['Washington','Clark'],['Washington','Crawford'],['Washington','Floyd'],['Washington','Harrison'],['Washington','Jackson'],['Washington','Lawrence'],['Washington','Orange'],['Washington','Scott'],['Wayne','Fayette'],['Wayne','Henry'],['Wayne','Randolph'],['Wayne','Union'],['Wells','Adams'],['Wells','Allen'],['Wells','Blackford'],['Wells','Grant'],['Wells','Huntington'],['Wells','Jay'],['White','Benton'],['White','Carroll'],['White','Cass'],['White','Jasper'],['White','Pulaski'],['White','Tippecanoe'],['Whitley','Allen'],['Whitley','Huntington'],['Whitley','Kosciusko'],['Whitley','Noble'],['Whitley','Wabash']]

In [6]:
df_pair = pd.DataFrame(county_pair_list, columns=['county','adj_county'])
#df_pair = df_pair.groupby('county')['adj_county'].apply(', '.join).reset_index()
#df_pair = df_pair.groupby('county')['adj_county'].apply(list).to_dict()

### Pre-Model Research

In [7]:
state_pop = df0['pop'].sum()
pop_per_rep = state_pop / 9 #gives us population per representative
f'State Pop Total is {state_pop}, number people per representive is {pop_per_rep}'

'State Pop Total is 6785668, number people per representive is 753963.1111111111'

In [8]:
#lets assume we are going to allow for variance off the average of 5% up or down
#how many counties do we have that exceed the upper population limit on their own
max_per_district = pop_per_rep * 1.05
df0[df0['pop'] >= max_per_district]['county']

48    Marion
Name: county, dtype: object

Marion county is the only county that by itself has a population that exceeds the average population per representitive.  We are not allowing for divisibility of counties in this problem, so we can either (1) assign two representatives to this county or (2) allow only one representitive which would result in under representation in Marion county.

Two representitive in Marion county would result in a population per representive in Marion of 488k per representive.  This would result in 488k people per representive in Marion county and 830k in the rest of the state.  That represents a substantial over-representation in Marion county.  Hence, we prefer the alternative under representation in Marion county

### Global Variables

In [9]:
num_counties = county_cnt #number of counties
num_leg_districits = 9 #number of legistlative districts
pop_max = 977213 #max pop allosed in each legislative district
pop_min = pop_per_rep / 1.05 #min pop allowed in each legislative district

### Setup

In [10]:
#creating a variable for each county.
counties = df0['county'].unique()

In [11]:
#creating a variable for each district
district = list(range(0,num_leg_districits))

In [12]:
#creating a population dictionary
pop_list = df0['pop']
pop = dict(zip(counties, pop_list))

In [13]:
#adjacency dictionary
adj_dict = df_pair.groupby('county')['adj_county'].apply(list).to_dict()

### Creating All Possible Adjacent Pairs

In [14]:
pair_temp = []
sorted_pair_temp = []
start_county = []
end_county = []
for i in counties:
    for p in adj_dict[i]:
       pair_temp.append(i+p)
       sorted_pair_temp.append(sorted(i+p))
       start_county.append(i)
       end_county.append(p)
d = {'pair_list':pair_temp,'sorted':sorted_pair_temp,"start":start_county,"end":end_county}
df = pd.DataFrame(d)
key_list = df.sorted.drop_duplicates().keys()
pair_list = df.pair_list[key_list]
start_list = df.start[key_list]
end_list = df.end[key_list]

### Creating a dataframe of each pair within each district layer

In [15]:
#this will help built the constraint a little quicker, so doing this here and now
pair_list2  = [(p,d) for p in pair_list for d in district]
start_list2  = [(s,d) for s in start_list for d in district]
end_list2  = [(e,d) for e in end_list for d in district]
d2 = {'pair_list':pair_list2,'start':start_list2,"end":end_list2}
df2 = pd.DataFrame(d2)

### Establishing Variables

In [16]:
#establish county/district variables
county_district = LpVariable.dicts("count_district_assignment",[(c,d) for c in counties for d in district],0,1,cat="Integer")
adj_pair = LpVariable.dicts("adj_pair",[(p,d) for p in pair_list for d in district],0,1,cat="Integer")

### Objective Function

In [17]:
#setting up problem
prob = LpProblem("problem",LpMinimize)
prob += lpSum([adj_pair[i] for i in adj_pair])

### Establishing constraints

In [18]:
#making sure each county is assigned to one and only one district
for c in counties:
    prob += lpSum([county_district[(c,d)] for d in district]) == 1
    
#setting up constraints on max population in each district
for d in district:
    prob += lpSum([county_district[(i,d)] * pop[i] for i in counties]) <= pop_max 

#setting up constraints on min population in each district
for d in district:
    prob += lpSum([county_district[(i,d)] * pop[i] for i in counties]) >= pop_min 
    
#counting edges and relating to the edge variables
for i in range(len(pair_list2)):
    prob +=  county_district[start_list2[i]] - county_district[end_list2[i]] <= adj_pair[pair_list2[i]]
    
for i in range(len(pair_list2)):
    prob +=  -county_district[start_list2[i]] + county_district[end_list2[i]] <= adj_pair[pair_list2[i]]

### Solve

In [ ]:
prob.solve()

### Show the Assignment of County to Legislative District

In [ ]:
assignment_list = []
assignment_county = []
assignment_district = []
for i in county_district: 
    assignment_list.append(county_district[i].varValue)
for c in counties:
    for j in district:
        assignment_county.append(c)
        assignment_district.append(j)
d3 = {"County":assignment_county, "Legislative_District": assignment_district, "assignment_list": assignment_list}
df3 = pd.DataFrame(d3)
df3 = df3[df3['assignment_list'] == 1]

In [ ]:
df3.groupby('Legislative_District')['County'].apply(' '.join).reset_index()

### Count the Number of Cut Edges

In [ ]:
edge_total = [] 
for i in pair_list2: 
    edge_total.append(adj_pair[i].varValue/2)
print('Total of Cut Edges is {}'.format(sum(edge_total)))